<a href="https://colab.research.google.com/github/MinuMaryS/Coursera_Capstone/blob/main/Italian_Restaurants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import pandas as pd
import numpy as np
import folium 
import requests # library to handle requests
import random
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [139]:
url="https://en.wikipedia.org/wiki/Regions_of_Italy"
df=pd.read_html(url,header=1)

In [140]:
 postalcodes=df[1]

In [141]:
postalcodes.head()

,Flag,RegionItalian name (if different),Status,Number,%,km2,%.1,Pop. density,HDI[4],Capital,President,President.1,Number of comuni[5],Prov. ormetrop. cities
0,NaN,Abruzzo,Ordinary,1311580,2.17%,10832,3.59%,121,0.890,L'Aquila,NaN,Marco MarsilioBrothers of Italy,305,4
1,NaN,Aosta ValleyValle d'Aosta,Autonomous,125666,0.21%,3261,1.08%,39,0.878,Aosta,NaN,Erik LavévazValdostan Union,74,1
2,NaN,ApuliaPuglia,Ordinary,4029053,6.68%,19541,6.48%,206,0.852,Bari,NaN,Michele EmilianoDemocratic Party,258,6
3,NaN,Basilicata,Ordinary,562869,0.93%,10073,3.34%,56,0.857,Potenza,NaN,Vito BardiForza Italia,131,2
4,NaN,Calabria,Ordinary,1947131,3.23%,15222,5.04%,128,0.850,Catanzaro,NaN,Antonino Spirlì (acting)[a]League,404,5


In [142]:
postalcodes.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Flag                               0 non-null      float64
 1   RegionItalian name (if different)  21 non-null     object 
 2   Status                             21 non-null     object 
 3   Number                             21 non-null     int64  
 4   %                                  21 non-null     object 
 5   km2                                21 non-null     int64  
 6   %.1                                21 non-null     object 
 7   Pop. density                       21 non-null     int64  
 8   HDI[4]                             21 non-null     float64
 9   Capital                            21 non-null     object 
 10  President                          0 non-null      float64
 11  President.1                        21 non-null     object 
 

In [143]:
postalcodes.columns

Index(['Flag', 'RegionItalian name (if different)', 'Status', 'Number', '%',
       'km2', '%.1', 'Pop. density', 'HDI[4]', 'Capital', 'President',
       'President.1', 'Number of comuni[5]', 'Prov. ormetrop. cities'],
      dtype='object')

In [144]:
postalcodes.rename(columns={'RegionItalian name (if\xa0different)':'Region'}, inplace=True)

In [145]:
postalcodes.Region.unique()

array(['Abruzzo', "Aosta ValleyValle d'Aosta", 'ApuliaPuglia',
       'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
       'Friuli Venezia Giulia', 'Lazio', 'Liguria', 'LombardyLombardia',
       'Marche', 'Molise', 'PiedmontPiemonte', 'SardiniaSardegna',
       'SicilySicilia', 'Trentino-South TyrolTrentino-Alto Adige',
       'TuscanyToscana', 'Umbria', 'Veneto', 'ItalyItalia'], dtype=object)

In [146]:
postalcodes['Region']=postalcodes.Region.str.replace("Aosta ValleyValle d'Aosta","Aosta Valley").str.replace("ApuliaPuglia",'Apulia')

In [147]:
postalcodes['Region']=postalcodes.Region.str.replace("LombardyLombardia","Lombardy").str.replace("PiedmontPiemonte",'Piedmont')
postalcodes['Region']=postalcodes.Region.str.replace("SardiniaSardegna","Sardinia").str.replace("SicilySicilia",'Sicily').str.replace("ItalyItalia",'Italy')
postalcodes['Region']=postalcodes.Region.str.replace("Trentino-South TyrolTrentino-Alto Adige","Trentino-South Tyrol").str.replace("TuscanyToscana",'Tuscany')

In [148]:

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize  to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Italy'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [149]:
coords = [ get_latlng(neighborhood) for neighborhood in postalcodes['Region'].tolist() ]


In [150]:
coords

[[42.22755559500007, 13.85504873900004],
 [45.73025715400007, 7.387197438000044],
 [40.98504105300003, 16.61878563600004],
 [40.499969168000064, 16.081552375000058],
 [39.06808148600004, 16.347669620000033],
 [40.85973489000003, 14.840116015000035],
 [44.52591909400007, 11.039218098000049],
 [46.15430985200004, 13.052973187000077],
 [41.97568202100007, 12.772625418000075],
 [44.415350258000046, 9.427370000000053],
 [45.61724995900005, 9.769085876000076],
 [43.34820021300004, 13.137824072000058],
 [41.684317099000054, 14.595198312000036],
 [45.057300686000076, 7.920492624000076],
 [40.08780103000004, 9.030487710000045],
 [37.50000000000006, 14.000000000000057],
 [44.238170000000025, 10.820530000000076],
 [43.45082962300006, 11.126187096000024],
 [42.96333536100008, 12.49542613500006],
 [45.65492497900004, 11.854320125000072],
 [42.833330000000046, 12.833330000000046]]

In [151]:
 df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [152]:

# merge the coordinates into the original dataframe
postalcodes['Latitude'] = df_coords['Latitude']
postalcodes['Longitude'] = df_coords['Longitude']

In [153]:
Italy=postalcodes[['Region','Latitude','Longitude']]

In [154]:
Italy

,Region,Latitude,Longitude
0,Abruzzo,42.227556,13.855049
1,Aosta Valley,45.730257,7.387197
2,Apulia,40.985041,16.618786
3,Basilicata,40.499969,16.081552
4,Calabria,39.068081,16.347670
5,Campania,40.859735,14.840116
6,Emilia-Romagna,44.525919,11.039218
7,Friuli Venezia Giulia,46.154310,13.052973
8,Lazio,41.975682,12.772625
9,Liguria,44.415350,9.427370


In [155]:
address = 'Italy'

geolocator = Nominatim(user_agent="my-appl")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Italy {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Italy 42.6384261, 12.674297.


In [156]:
 # create map of Italy using latitude and longitude values
map_itl = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, neighborhood in zip(Italy['Latitude'], Italy['Longitude'], Italy['Region']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_itl)  
    
map_itl

In [189]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [158]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [159]:
radius = 10000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(Italy['Latitude'], Italy['Longitude'], Italy['Region']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [160]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(733, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Abruzzo,42.227556,13.855049,Abbazia di San Clemente a Casauria,42.235114,13.928845,Historic Site
1,Abruzzo,42.227556,13.855049,A25 Casello Torre Dei Passeri-Casauria,42.236525,13.924055,Toll Booth
2,Abruzzo,42.227556,13.855049,Parco Del Sole,42.172878,13.831728,Park
3,Abruzzo,42.227556,13.855049,Stazione FS Popoli-Vittorito,42.172731,13.825840,Train Station
4,Abruzzo,42.227556,13.855049,Riserva Naturale Sorgenti del Pescara,42.166966,13.821316,Scenic Lookout


In [161]:
venues_df.groupby(["Neighborhood"])['VenueCategory'].count()


Neighborhood
Abruzzo                    5
Aosta Valley              76
Apulia                     4
Basilicata                 4
Calabria                   7
Campania                  38
Emilia-Romagna            81
Friuli Venezia Giulia     55
Italy                      9
Lazio                     60
Liguria                    7
Lombardy                 100
Marche                     8
Molise                     4
Piedmont                  33
Sardinia                   4
Sicily                     4
Trentino-South Tyrol      15
Tuscany                  100
Umbria                    22
Veneto                    97
Name: VenueCategory, dtype: int64

In [162]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))


There are 133 uniques categories.


In [163]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Historic Site', 'Toll Booth', 'Park', 'Train Station',
       'Scenic Lookout', 'Boarding House', 'Hotel', 'Restaurant',
       'Italian Restaurant', 'Cocktail Bar', 'Aosta Restaurant',
       'Ski Area', 'Wine Bar', 'Plaza', 'Dessert Shop', 'Ice Cream Shop',
       'Pub', 'Asian Restaurant', 'Museum', 'Pizza Place', 'Café',
       "Men's Store", 'Ski Chairlift', 'Trattoria/Osteria', 'Shoe Store',
       'Castle', 'Church', 'Japanese Restaurant', 'Diner',
       'Apres Ski Bar', 'Agriturismo', 'Supermarket',
       'Vegetarian / Vegan Restaurant', 'Steakhouse', 'Multiplex',
       'Electronics Store', 'Grocery Store', 'Seafood Restaurant',
       'Fast Food Restaurant', 'Furniture / Home Store', 'Skate Park',
       'Ski Lodge', 'Auto Garage', 'Theme Park Ride / Attraction',
       'Mountain', 'Bar', 'Beer Garden', 'Outdoors & Recreation',
       'Tea Room', 'Cupcake Shop'], dtype=object)

In [164]:
venues_df['VenueCategory'].value_counts()

Italian Restaurant     133
Pizza Place             52
Restaurant              50
Hotel                   47
Café                    33
                      ... 
American Restaurant      1
Basketball Stadium       1
Boarding House           1
Frozen Yogurt Shop       1
Gas Station              1
Name: VenueCategory, Length: 133, dtype: int64

In [165]:
it_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
it_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [it_onehot.columns[-1]] + list(it_onehot.columns[:-1])
it_onehot = it_onehot[fixed_columns]

print(it_onehot.shape)
it_onehot.head()

(733, 134)


,Neighborhoods,Agriturismo,American Restaurant,Aosta Restaurant,Apres Ski Bar,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boarding House,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Castle,Cheese Shop,Chinese Restaurant,Church,City,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,...,Pool,Pub,Racetrack,Rental Service,Resort,Rest Area,Restaurant,Rock Club,Sandwich Place,Sardinian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Area,Ski Chairlift,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Theme Park Ride / Attraction,Toll Booth,Trail,Train Station,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Veneto Restaurant,Wine Bar,Wine Shop,Winery
0,Abruzzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abruzzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,Abruzzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Abruzzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Abruzzo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [166]:
it_grouped = it_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(it_grouped.shape)
it_grouped

(21, 134)


,Neighborhoods,Agriturismo,American Restaurant,Aosta Restaurant,Apres Ski Bar,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boarding House,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Castle,Cheese Shop,Chinese Restaurant,Church,City,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,...,Pool,Pub,Racetrack,Rental Service,Resort,Rest Area,Restaurant,Rock Club,Sandwich Place,Sardinian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Ski Area,Ski Chairlift,Ski Lodge,Snack Place,Soccer Field,Soccer Stadium,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Theme Park Ride / Attraction,Toll Booth,Trail,Train Station,Trattoria/Osteria,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Veneto Restaurant,Wine Bar,Wine Shop,Winery
0,Abruzzo,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.200000,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
1,Aosta Valley,0.013158,0.00,0.013158,0.026316,0.00,0.00,0.00,0.000000,0.013158,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.065789,0.013158,0.000000,0.000000,0.013158,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.00,...,0.000000,0.039474,0.000000,0.00,0.00,0.000000,0.052632,0.000000,0.000000,0.00,0.000000,0.00,0.013158,0.013158,0.000000,0.013158,0.013158,0.013158,0.013158,0.000000,0.000000,0.000000,0.000000,0.00,0.013158,0.039474,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.026316,0.00,0.013158,0.000000,0.013158,0.000000,0.000000
2,Apulia,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.25,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Basilicata,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Calabria,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.142857,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [167]:
postalcodes.rename(columns={'Region':'Neighborhood'}, inplace=True)

In [168]:
Italianrest=it_grouped[['Neighborhoods','Italian Restaurant']]

In [169]:
Italianrest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Neighborhoods       21 non-null     object 
 1   Italian Restaurant  21 non-null     float64
dtypes: float64(1), object(1)
memory usage: 464.0+ bytes


In [170]:
Italianrest

,Neighborhoods,Italian Restaurant
0,Abruzzo,0.000000
1,Aosta Valley,0.105263
2,Apulia,0.000000
3,Basilicata,0.000000
4,Calabria,0.142857
5,Campania,0.105263
6,Emilia-Romagna,0.370370
7,Friuli Venezia Giulia,0.181818
8,Italy,0.333333
9,Lazio,0.166667


In [171]:
# set number of clusters
kclusters = 3

kl_clustering = Italianrest.drop(columns=["Neighborhoods"])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
clusters=kmeans.labels_[0:21]

In [172]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = Italianrest.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [173]:

kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged

,Neighborhood,Italian Restaurant,Cluster Labels
0,Abruzzo,0.000000,1
1,Aosta Valley,0.105263,2
2,Apulia,0.000000,1
3,Basilicata,0.000000,1
4,Calabria,0.142857,2
5,Campania,0.105263,2
6,Emilia-Romagna,0.370370,0
7,Friuli Venezia Giulia,0.181818,2
8,Italy,0.333333,0
9,Lazio,0.166667,2


In [174]:
Italy.rename(columns={"Region": "Neighborhood"}, inplace=True)
Italy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Neighborhood,Latitude,Longitude
0,Abruzzo,42.227556,13.855049
1,Aosta Valley,45.730257,7.387197
2,Apulia,40.985041,16.618786
3,Basilicata,40.499969,16.081552
4,Calabria,39.068081,16.347670
5,Campania,40.859735,14.840116
6,Emilia-Romagna,44.525919,11.039218
7,Friuli Venezia Giulia,46.154310,13.052973
8,Lazio,41.975682,12.772625
9,Liguria,44.415350,9.427370


In [175]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Italy_cluster = kl_merged.join(Italy.set_index("Neighborhood"), on="Neighborhood")

#print(kl_merged.shape)
Italy_cluster.head() # check the last columns!

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Abruzzo,0.000000,1,42.227556,13.855049
1,Aosta Valley,0.105263,2,45.730257,7.387197
2,Apulia,0.000000,1,40.985041,16.618786
3,Basilicata,0.000000,1,40.499969,16.081552
4,Calabria,0.142857,2,39.068081,16.347670


In [178]:
# sort the results by Cluster Labels
print(Italy_cluster.shape)
Italy_cluster.sort_values(["Cluster Labels"], inplace=True)
Italy_cluster

(21, 5)


,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
10,Liguria,0.285714,0,44.415350,9.427370
17,Trentino-South Tyrol,0.266667,0,44.238170,10.820530
12,Marche,0.250000,0,43.348200,13.137824
6,Emilia-Romagna,0.370370,0,44.525919,11.039218
19,Umbria,0.454545,0,42.963335,12.495426
8,Italy,0.333333,0,42.833330,12.833330
16,Sicily,0.000000,1,37.500000,14.000000
15,Sardinia,0.000000,1,40.087801,9.030488
13,Molise,0.000000,1,41.684317,14.595198
0,Abruzzo,0.000000,1,42.227556,13.855049


In [179]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Italy_cluster['Latitude'], Italy_cluster['Longitude'], Italy_cluster['Neighborhood'], Italy_cluster['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [188]:
Italy_cluster[Italy_cluster['Cluster Labels']==0]

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
10,Liguria,0.285714,0,44.415350,9.427370
17,Trentino-South Tyrol,0.266667,0,44.238170,10.820530
12,Marche,0.250000,0,43.348200,13.137824
6,Emilia-Romagna,0.370370,0,44.525919,11.039218
19,Umbria,0.454545,0,42.963335,12.495426
8,Italy,0.333333,0,42.833330,12.833330


In [187]:
Italy_cluster[Italy_cluster['Cluster Labels']==1]

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
16,Sicily,0.0,1,37.500000,14.000000
15,Sardinia,0.0,1,40.087801,9.030488
13,Molise,0.0,1,41.684317,14.595198
0,Abruzzo,0.0,1,42.227556,13.855049
3,Basilicata,0.0,1,40.499969,16.081552
2,Apulia,0.0,1,40.985041,16.618786


In [186]:
Italy_cluster[Italy_cluster['Cluster Labels']==2]

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
7,Friuli Venezia Giulia,0.181818,2,46.154310,13.052973
11,Lombardy,0.120000,2,45.617250,9.769086
5,Campania,0.105263,2,40.859735,14.840116
4,Calabria,0.142857,2,39.068081,16.347670
14,Piedmont,0.121212,2,45.057301,7.920493
1,Aosta Valley,0.105263,2,45.730257,7.387197
18,Tuscany,0.200000,2,43.450830,11.126187
9,Lazio,0.166667,2,41.975682,12.772625
20,Veneto,0.134021,2,45.654925,11.854320
